### Task 1:  Load the Dane County ZCTA polygons and overlay on Google Earth Engine BaseMap

In [2]:
import ee
ee.Authenticate()

Enter verification code: 4/1AVHEtk7cjssIL1_8ZtLIu95IWc8u35wVYqNNNj0RFpbtfIcV4sxqw7L4WWU

Successfully saved authorization token.


In [13]:
import geemap
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [100]:
#Adding map to display
Map = geemap.Map()
#Adding Dane county ZCTA shapefile
dane_shp = r'D:\CourseWork_GIS_2022\Geog 573 - Geospatial Big data Analytics\Lab3\ZCTAs_DaneCounty\ZCTAs_DaneCounty.shp'
dane_zcta = geemap.shp_to_ee(dane_shp)
Map.setCenter(-89.40091425818218,43.07585698391842, 9)
style = {'color': '#993404', 'width': 1, 'lineType': 'solid', 'fillColor': '#fec44f'}
Map.add_basemap("Esri.WorldGrayCanvas") #ESRI Gray base map
Map.addLayer(dane_zcta.style(**style), {}, 'Dane Zipcodes')
Map

Map(center=[43.07585698391842, -89.40091425818218], controls=(WidgetControl(options=['position', 'transparent_…

### Task 2: Calculate Zonal Statitics for Dane County (2016, 2022)

In [27]:
import ee
import geemap

Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [28]:
# Setting the region of interest using map extent (NE, SW values)
region = Map.user_roi
if region is None:
    #Dane county North east, South west coordinates
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)
Map.centerObject(region)

In [29]:
# Set the date range for query
start_date = '2016-01-01'
end_date = '2016-12-31'

In [30]:
# Dynamic World land cover composite
landcover = geemap.dynamic_world(region, start_date, end_date, return_type = 'hillshade')
Map.addLayer(landcover, {}, 'Land Cover')

# Add legend to the map
Map.add_legend(title = 'Dynamic World Land Cover', builtin_legend = 'Dynamic_World')
Map.centerObject(region)
Map

Map(bottom=96556.0, center=[43.068887774169625, -89.296875], controls=(WidgetControl(options=['position', 'tra…

In [35]:
## Return "class" images for further zonal summary analysis 
classes = geemap.dynamic_world(region, start_date, end_date, return_type='class')
df_2016 = geemap.image_area_by_group(classes, region=region, scale=10, denominator=1e6)
df_2016

Calculating area for group 0 ...
Calculating area for group 1 ...
Calculating area for group 2 ...
Calculating area for group 3 ...
Calculating area for group 4 ...
Calculating area for group 5 ...
Calculating area for group 6 ...
Calculating area for group 7 ...
Calculating area for group 8 ...


,area,percentage
group,,
0,93.0558,0.0505
1,465.8921,0.2528
2,131.8860,0.0716
3,2.8323,0.0015
4,771.4642,0.4187
5,11.9282,0.0065
6,355.4726,0.1929
7,7.9090,0.0043
8,2.1540,0.0012


In [36]:
# Set the date range to query the data
start_date = '2022-01-01'
end_date = '2022-12-31'

In [61]:
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')
Map.addLayer(landcover, {}, 'Land Cover')

In [38]:
## Return "class" images for further zonal summary analysis
classes = geemap.dynamic_world(region, start_date, end_date, return_type='class')
df_2022 = geemap.image_area_by_group(classes, region=region, scale=10, denominator=1e6)
df_2022

Calculating area for group 0 ...
Calculating area for group 1 ...
Calculating area for group 2 ...
Calculating area for group 3 ...
Calculating area for group 4 ...
Calculating area for group 5 ...
Calculating area for group 6 ...
Calculating area for group 7 ...
Calculating area for group 8 ...


,area,percentage
group,,
0,93.0558,0.0505
1,465.8921,0.2528
2,131.8860,0.0716
3,2.8323,0.0015
4,771.4642,0.4187
5,11.9282,0.0065
6,355.4726,0.1929
7,7.9090,0.0043
8,2.1540,0.0012


In [47]:
df_2016 = df_2016.rename(columns = {'percentage':'percentage_2016', 'area':'area_2016'})
df_2016.head(1)

,area_2016,percentage_2016
group,,
0,93.0558,0.0505


In [48]:
df_2022 = df_2022.rename(columns = {'percentage':'percentage_2022', 'area':'area_2022'})
df_2022.head(1)

,area_2022,percentage_2022
group,,
0,93.0558,0.0505


In [55]:
import pandas as pd
list_types = ['water','Trees','Grass','Flooded vegetation','crops','Shrub & Scrub','Built-up area','Bare ground','Snow & Ice']
zonal_stats_df = pd.DataFrame(
    {'group' : list_types,
    'area_2016' : list(df_2016['area_2016']),
    'percentage_2016' : list(df_2016['percentage_2016']),
    'area_2022' : list(df_2022['area_2022']),
    'percentage_2022' : list(df_2022['percentage_2022'])
    })

In [59]:
zonal_stats_df['% change'] = zonal_stats_df.apply(lambda x : (x['percentage_2016'] - x['percentage_2022']), axis = 1)

In [60]:
zonal_stats_df

,group,area_2016,percentage_2016,area_2022,percentage_2022,% change
0,water,93.0558,0.0505,93.0558,0.0505,0.0
1,Trees,465.8921,0.2528,465.8921,0.2528,0.0
2,Grass,131.8860,0.0716,131.8860,0.0716,0.0
3,Flooded vegetation,2.8323,0.0015,2.8323,0.0015,0.0
4,crops,771.4642,0.4187,771.4642,0.4187,0.0
5,Shrub & Scrub,11.9282,0.0065,11.9282,0.0065,0.0
6,Built-up area,355.4726,0.1929,355.4726,0.1929,0.0
7,Bare ground,7.9090,0.0043,7.9090,0.0043,0.0
8,Snow & Ice,2.1540,0.0012,2.1540,0.0012,0.0


Between 2016 and 2022, there are no changes visible using Dynamic World data

In [90]:
zonal_df = zonal_stats_df.drop(['area_2016', 'area_2022', '% change'], axis = 1)
zonal_df = zonal_df.rename(columns = {'group':'classes'})
zonal_df.index.name = ''
zonal_stats_dict = zonal_df.to_dict()

In [91]:
#Required dictionary for two time periods based on class is published
zonal_stats_dict

{'classes': {0: 'water',
  1: 'Trees',
  2: 'Grass',
  3: 'Flooded vegetation',
  4: 'crops',
  5: 'Shrub & Scrub',
  6: 'Built-up area',
  7: 'Bare ground',
  8: 'Snow & Ice'},
 'percentage_2016': {0: 0.0505,
  1: 0.2528,
  2: 0.0716,
  3: 0.0015,
  4: 0.4187,
  5: 0.0065,
  6: 0.1929,
  7: 0.0043,
  8: 0.0012},
 'percentage_2022': {0: 0.0505,
  1: 0.2528,
  2: 0.0716,
  3: 0.0015,
  4: 0.4187,
  5: 0.0065,
  6: 0.1929,
  7: 0.0043,
  8: 0.0012}}

### Task 3: Normalized Difference Vegetation Index (NDVI)

In [68]:
#Input image from Landsat for study area (Fremont, California)
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
Map = geemap.Map()
#Zoom level 9 | Higher the value, more the zoom
Map.centerObject(image, 9)
#Selecting B4 - Green, B5 - Red, B6 - Near Infrared (NIR) for display
Map.addLayer(image, {'bands': ['B6', 'B5', 'B4']}, 'Landsat 8')
Map

Map(center=[37.471602850590656, -122.14451407746262], controls=(WidgetControl(options=['position', 'transparen…

In [69]:
#Selecting NIR, Red bands from Lansat 8 imagery
NIR = image.select('B5')
Red = image.select('B4')

#Formula NDVI = (NIR band - Red band) / (NIR band + Red band)
NDVI = NIR.subtract(Red).divide(NIR.add(Red))

#color palette to display the NDVI for the study area (from GEE Map)
palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

#Adding it as a map layer with palette and annotation
Map.addLayer(NDVI, {'palette': palette}, "NDVI Landsat 8")
#displaying the layer on the map
Map

Map(bottom=51101.0, center=[37.471602850590656, -122.14451407746262], controls=(WidgetControl(options=['positi…

### Task 4: Landsat Timeline Animation GIF

In [79]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
# Region of interest is Las Vegas, Nevada (Whole extent is considered for analysis)
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-115.3618, 36.3760, -114.8327, 35.9659)
Map.centerObject(region)
# Set the date range
start_date = '1985-01-01'
end_date = '2022-12-31'
#images are taken from Dynamic World with quaterly frequency
images = geemap.dynamic_world_timeseries(
    region, start_date, end_date, frequency="quarter", return_type="hillshade"
)
#images are added to map
Map.addLayer(images)
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [80]:
#Adding map and basemap
Map = geemap.Map()
Map.add_basemap('HYBRID')
#Centering the map over the region
Map.centerObject(region)
#Adding juxtapose legend between 07/15 to 12/22
Map.ts_inspector(images, date_format='YYYY-MM')
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
#Map display
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Lay…

In [101]:
import os

#Adding map
Map = geemap.Map()
#Adding title to GIF
label = 'Land Cover/Use Changes in Las Vegas, NV'

#Using OS to find the downloads folder to append GIF
path = os.path.join(os.path.expanduser('~'), 'downloads')

#Defining parameters to render GIF
Map.add_landsat_ts_gif(
    label=label,
    roi=region,
    start_year=1985,
    bands=['Red', 'Green', 'Blue'],
    font_color='white',
    frames_per_second=10,
    progress_bar_color='#4d9221',
    out_gif= path +'/landsat_ts.gif',
)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\Sidrcs\downloads\landsat_ts.gif
ffmpeg is not installed on your computer.
Adding GIF to the map ...
The timelapse has been added to the map.
